## 원티드 홈피에서 키워드 검색 후 포지션 명, 회사명, 포지션 상세URL 수집

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
from bs4 import BeautifulSoup

def crawl_wanted_llm():
    # 1) 드라이버 세팅
    CHROMEDRIVER_PATH = r"./data/chromedriver.exe"
    service = Service(CHROMEDRIVER_PATH)
    driver = webdriver.Chrome(service=service)

    try:
        # 2) "포지션 탭" 직링크 시도 (url에서 쿼리와 tab 사이에 키워드를 입력하면 됨.)
        position_tab_url = "https://www.wanted.co.kr/search?query=인공지능&tab=position"
        driver.get(position_tab_url)
        time.sleep(2)

        # 3) 혹시 공고가 제대로 뜨지 않는다면, "포지션 전체보기" 버튼 클릭 로직을 백업으로
        #    (필요하다면)
        #    아래는 예시이며, 실제로 이 로직이 필요 없을 수도 있습니다.
        # =============================================================
        # try:
        #     wait = WebDriverWait(driver, 10)
        #     button_locator = (By.CSS_SELECTOR, "button[data-testid='SearchContentViewMoreButton']")
        #     view_all_button = wait.until(EC.element_to_be_clickable(button_locator))
        #
        #     # 스크롤로 버튼 노출
        #     driver.execute_script("arguments[0].scrollIntoView(true);", view_all_button)
        #     time.sleep(1)
        #     # JS 강제 클릭 (겹치는 요소 방지)
        #     driver.execute_script("arguments[0].click();", view_all_button)
        #     time.sleep(2)
        # except:
        #     pass
        # =============================================================

        # 4) 무한 스크롤 (position 탭에 들어왔다고 가정)
        same_count_times = 0
        while True:
            # 4-1) 현재 공고 수
            current_cards = driver.find_elements(By.CSS_SELECTOR, "a[data-position-name][data-company-name]")
            current_count = len(current_cards)

            # 4-2) window 전체 스크롤 (원티드에서 body 스크롤이면)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

            # 4-3) 새 공고 수
            new_cards = driver.find_elements(By.CSS_SELECTOR, "a[data-position-name][data-company-name]")
            new_count = len(new_cards)

            # 4-4) 변화 체크
            if new_count == current_count:
                same_count_times += 1
            else:
                same_count_times = 0

            if same_count_times >= 2:
                break

        # 5) HTML 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        job_cards = soup.find_all("a", {
            "data-position-name": True,
            "data-company-name": True
        })

        results = []
        for card in job_cards:
            p_name = card.get("data-position-name", "").strip()
            c_name = card.get("data-company-name", "").strip()
            href = card.get("href", "")

            if href.startswith("/"):
                href = "https://www.wanted.co.kr" + href

            results.append([p_name, c_name, href])

        # 6) CSV 저장
        with open("wanted_ingong_jobs.csv", "w", encoding="utf-8", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["공고명", "회사명", "URL"])
            for row in results:
                writer.writerow(row)

        print(f"수집 완료. 총 {len(results)}개 공고")

    finally:
        driver.quit()


if __name__ == "__main__":
    crawl_wanted_llm()


수집 완료. 총 56개 공고


In [4]:
import pandas as pd

# 파일 불러오기
file_path = "wanted_ingong_jobs.csv" 
df = pd.read_csv(file_path, encoding="utf-8")

# 중복 데이터 제거
df_cleaned = df.drop_duplicates()

# 중복이 제거된 데이터 저장
output_file = "wanted_ingong_jobs_cleaned.csv"
df_cleaned.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"중복 제거 완료! 파일 저장: {output_file}")


중복 제거 완료! 파일 저장: wanted_ingong_jobs_cleaned.csv


In [5]:
import pandas as pd
import glob  # 파일 목록 가져오는 라이브러리

# 1️⃣ 병합할 CSV 파일 리스트 가져오기 (현재 폴더에 있는 파일 기준)
file_list = glob.glob("wanted_*.csv")  

# 2️⃣ CSV 파일을 하나의 DataFrame으로 병합
df_list = [pd.read_csv(file, encoding="utf-8") for file in file_list]
df_merged = pd.concat(df_list, ignore_index=True)  # 인덱스 초기화

# 3️⃣ 중복 데이터 제거
df_final = df_merged.drop_duplicates()

# 4️⃣ 새로운 파일로 저장
output_file = "wanted_merged.csv"
df_final.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✅ 파일 병합 및 중복 제거 완료! 저장된 파일: {output_file}")


✅ 파일 병합 및 중복 제거 완료! 저장된 파일: wanted_merged.csv


In [3]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('wanted_merged.csv')

# 1~3행을 제외한 나머지 행 삭제
df = df.head(3)

# 수정된 DataFrame을 CSV 파일로 저장
df.to_csv('wanted_merged(1).csv', index=False)


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def scrape_wanted_job_descriptions(csv_file_path, output_file_path):
    """
    원티드에서 각 공고 URL로 들어가서 상세 정보 및 기술 스택을 크롤링하는 함수
    1. CSV에서 각 공고 URL을 읽음
    2. 각 페이지에 접속하여 Job Description + 기술 스택 수집
    3. '상세 정보 더 보기' 버튼이 있으면 클릭하여 전체 내용을 수집
    4. 기술 스택이 존재하는 경우 함께 저장
    """

    # ✅ ChromeDriver 경로 설정 (사용 환경에 맞게 수정)
    CHROMEDRIVER_PATH = r"./data/chromedriver.exe"
    service = Service(CHROMEDRIVER_PATH)
    driver = webdriver.Chrome(service=service)

    # ✅ CSV 파일 읽기
    df = pd.read_csv(csv_file_path)

    # ✅ 새로운 컬럼 추가 (경력, Job Description, 기술 스택)
    df["Experience"] = ""
    df["JobDescription"] = ""
    df["JobSkill"] = ""

    try:
        for idx, row in df.iterrows():
            job_url = row["URL"]  # CSV에 저장된 URL 열의 이름이 "URL"이어야 함
            
            print(f"Scraping {idx + 1}/{len(df)}: {job_url}")

            # ✅ 해당 URL로 이동
            driver.get(job_url)
            time.sleep(2)  # 페이지 로딩 대기

            wait = WebDriverWait(driver, 10)

            # ✅ (1) '경력' 정보 수집
            try:
                experience_elements = driver.find_elements(By.CSS_SELECTOR, "span.JobHeader_JobHeader__Tools__Company__Info__yT4OD")
                
                if len(experience_elements) > 1:  # 두 번째 요소가 존재하는 경우
                    experience = experience_elements[1].text.strip()
                    df.at[idx, "Experience"] = experience
                    print(f"✅ 경력 정보 수집 완료: {experience}")
                else:
                    df.at[idx, "Experience"] = "N/A"
                    print("❌ 경력 정보 없음")
            except Exception as e:
                print(f"❌ 경력 정보 수집 실패: {e}")
                df.at[idx, "Experience"] = "ERROR"

            # ✅ (2) '상세 정보 더 보기' 버튼이 있으면 클릭
            try:
                more_button_locator = (By.XPATH, "//button[span[contains(text(), '상세 정보 더 보기')]]")
                more_button = wait.until(EC.element_to_be_clickable(more_button_locator))
                
                # 버튼이 존재하면 클릭
                driver.execute_script("arguments[0].click();", more_button)
                time.sleep(2)  # 버튼 클릭 후 로딩 대기
                print("✅ '상세 정보 더 보기' 버튼 클릭 완료!")
            except:
                print("❌ '상세 정보 더 보기' 버튼 없음 (이미 전체 내용이 표시됨)")

            # ✅ (3) Job Description 제목 + 본문 내용 수집
            try:
                job_desc_container = driver.find_element(By.CSS_SELECTOR, "article.JobDescription_JobDescription__dq8G5")
                
                # ✅ (3-1) h2 (포지션 상세 제목) 수집
                try:
                    h2_element = job_desc_container.find_element(By.CSS_SELECTOR, "h2.wds-qf1364")
                    job_description_list = [h2_element.text.strip()]  # 리스트로 저장
                except:
                    job_description_list = []

                # ✅ (3-2) h3 (소제목) 수집
                h3_elements = job_desc_container.find_elements(By.CSS_SELECTOR, "h3.wds-l9subvB")
                job_description_list.extend([h3.text.strip() for h3 in h3_elements if h3.text.strip()])

                # ✅ (3-3) 본문 내용 (span.wds-wcfcu3) 수집
                span_elements = job_desc_container.find_elements(By.CSS_SELECTOR, "span.wds-wcfcu3")
                job_description_list.extend([span.text.strip() for span in span_elements if span.text.strip()])

                # ✅ 중복 제거 및 정렬
                unique_descriptions = list(set(job_description_list))
                unique_descriptions.sort()  # 순서 정리

                # ✅ 최종 문자열 변환 (줄바꿈 추가)
                job_description = "\n".join(unique_descriptions)

                df.at[idx, "JobDescription"] = job_description
                print(f"✅ Job Description 수집 완료 ({len(unique_descriptions)} 개 항목)")
            except Exception as e:
                print(f"❌ Job Description 수집 실패: {e}")
                df.at[idx, "JobDescription"] = "ERROR"

            # ✅ (4) 기술 스택 (JobSkill) 수집 (있는 경우만)
            try:
                job_skill_container = driver.find_element(By.CSS_SELECTOR, "article.JobSkillTags_JobSkillTags__UAOs6")
                
                # ✅ (4-1) h2 제목 수집 (기술 스택 · 툴)
                try:
                    h2_element = job_skill_container.find_element(By.CSS_SELECTOR, "h2.wds-qi3w5p")
                    job_skill_list = [h2_element.text.strip()]
                except:
                    job_skill_list = []

                # ✅ (4-2) 기술 스택 리스트(span.wds-1m3gvmz)
                skill_elements = job_skill_container.find_elements(By.CSS_SELECTOR, "span.wds-1m3gvmz")
                job_skill_list.extend([skill.text.strip() for skill in skill_elements if skill.text.strip()])

                # ✅ 중복 제거 및 정렬
                unique_skills = list(set(job_skill_list))
                unique_skills.sort()

                # ✅ 최종 문자열 변환 (쉼표로 구분)
                job_skill = ", ".join(unique_skills)

                df.at[idx, "JobSkill"] = job_skill
                print(f"✅ 기술 스택 수집 완료 ({len(unique_skills)} 개 항목)")
            except:
                print("❌ 기술 스택 없음")
                df.at[idx, "JobSkill"] = "N/A"

    finally:
        driver.quit()

        # ✅ (5) CSV로 저장
        df.to_csv(output_file_path, index=False, encoding="utf-8-sig")
        print(f"✅ 크롤링 완료! 저장된 파일: {output_file_path}")

# ✅ 실행
input_csv = "wanted_merged.csv"  # 기존 CSV 파일 (URL 포함)
output_csv = "wanted_ds_skil.csv"  # 저장할 파일

scrape_wanted_job_descriptions(input_csv, output_csv)


Scraping 1/537: https://www.wanted.co.kr/wd/270338
✅ 경력 정보 수집 완료: 경력 2-20년
✅ '상세 정보 더 보기' 버튼 클릭 완료!
✅ Job Description 수집 완료 (6 개 항목)
❌ 기술 스택 없음
Scraping 2/537: https://www.wanted.co.kr/wd/263703
✅ 경력 정보 수집 완료: 경력 3-10년
✅ '상세 정보 더 보기' 버튼 클릭 완료!
✅ Job Description 수집 완료 (6 개 항목)
❌ 기술 스택 없음
Scraping 3/537: https://www.wanted.co.kr/wd/268265
✅ 경력 정보 수집 완료: 경력 3-7년
✅ '상세 정보 더 보기' 버튼 클릭 완료!
✅ Job Description 수집 완료 (6 개 항목)
❌ 기술 스택 없음
Scraping 4/537: https://www.wanted.co.kr/wd/269317
✅ 경력 정보 수집 완료: 경력 2-7년
✅ '상세 정보 더 보기' 버튼 클릭 완료!
✅ Job Description 수집 완료 (6 개 항목)
❌ 기술 스택 없음
Scraping 5/537: https://www.wanted.co.kr/wd/269533
✅ 경력 정보 수집 완료: 경력 3년 이상
✅ '상세 정보 더 보기' 버튼 클릭 완료!
✅ Job Description 수집 완료 (6 개 항목)
❌ 기술 스택 없음
Scraping 6/537: https://www.wanted.co.kr/wd/270293
✅ 경력 정보 수집 완료: 경력 3년 이상
✅ '상세 정보 더 보기' 버튼 클릭 완료!
✅ Job Description 수집 완료 (6 개 항목)
❌ 기술 스택 없음
Scraping 7/537: https://www.wanted.co.kr/wd/269320
✅ 경력 정보 수집 완료: 경력 5-35년
✅ '상세 정보 더 보기' 버튼 클릭 완료!
✅ Job Description 수집 완료 (6 개 항목)
❌ 기술 